In [1]:
import sqlite3
import pandas as pd

### Section I 

In [2]:
conn = sqlite3.connect('cats.db')
c = conn.cursor()
c.execute('ATTACH DATABASE "dogs.db" AS db_dogs')
c.execute('ATTACH DATABASE "owners.db" AS db_owners')
# c.execute("""select * from cats join dogs on cats.Owner_ID = dogs.Owner_ID where dogs.Owner_ID=3;""")
c.execute("""SELECT owners_with_dogs.Name AS owner_name
, dogs.Name AS dog_name
, cats.Name AS cat_name 
FROM owners as owners_with_dogs 
JOIN dogs on owners_with_dogs.id = dogs.Owner_ID 
JOIN cats on owners_with_dogs.id = cats.Owner_ID
WHERE owner_name = 'Avi Flombaum'; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,owner_name,dog_name,cat_name
0,Avi Flombaum,Rex,Jackson


### Section II - 2 possible solutions 

#### a.

In [38]:
c.execute("""SELECT house_number, count((house_number)) as count FROM 
(SELECT * FROM dogs
UNION 
SELECT * FROM cats) AS AB
WHERE house_number IS NOT '' 
GROUP BY house_number
ORDER BY count DESC; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,House_Number,count
0,1,3
1,2,3
2,3,2
3,4,2
4,5,1


# b.

In [12]:
c.execute("""WITH dogs_count AS (SELECT house_number, count(name) AS num_dogs
FROM dogs 
GROUP BY house_number), cats_count AS 
(SELECT house_number, count(name) AS num_cats
FROM cats 
GROUP BY house_number)
SELECT dogs_count.house_number, num_dogs + num_cats as sum_pets FROM dogs_count
LEFT JOIN cats_count on dogs_count.house_number = cats_count.house_number
ORDER BY sum_pets DESC
LIMIT 2
; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,house_number,sum_pets
0,1,3
1,2,3


### Section III

In [45]:
c.execute("""SELECT owners_with_dogs.Name AS owner_name
, dogs.Name AS dog_name
, cats.Name AS cat_name
, owners_with_dogs.address
, owners_with_dogs.phone
FROM owners as owners_with_dogs 
JOIN dogs on owners_with_dogs.id = dogs.Owner_ID 
JOIN cats on owners_with_dogs.id = cats.Owner_ID
WHERE address = 'None'
GROUP BY owner_name, address
; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,owner_name,dog_name,cat_name,Address,Phone
0,Alison Peebles Madigan,Buddy,Rocky,None,2147098190
1,Justin Bieber,Bobo,Bell,None,4103810987
